In [12]:
import requests
import pandas as pd
from datetime import timedelta
from datetime import datetime

from airflow import DAG
from airflow.operators.python import PythonOperator

In [13]:

TOP_1M_DOMAINS = 'http://s3.amazonaws.com/alexa-static/top-1m.csv.zip'
TOP_1M_DOMAINS_FILE = 'top-1m.csv'


def get_data():
    # Здесь пока оставили запись в файл, как передавать переменую между тасками будет в третьем уроке
    top_doms = pd.read_csv(TOP_1M_DOMAINS)
    top_data = top_doms.to_csv(index=False)

    with open(TOP_1M_DOMAINS_FILE, 'w') as f:
        f.write(top_data)


def get_top_10():
    df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    df['length'] = df.apply(lambda row: len(row.domain), axis=1)
    res = df.sort_values(['length','domain'], ascending=[False,True]).iloc[0].domain
    with open('data_top_10.csv', 'w') as f:
        f.write(res)

def get_longest_name():
    df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    df['length'] = df.apply(lambda row: len(row.domain), axis=1)
    res = df.sort_values(['length','domain'], ascending=[False,True]).iloc[0].domain
    with open('longest_name.csv', 'w') as f:
        f.write(res)
                       
def get_airflow_position():
    df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    result = df[df.domain == 'airflow.com'].iloc[0]['rank']
    with open('airflow_position.csv', 'w') as f:
        f.write(str(result))


def print_data(ds):
    with open('data_top_10.csv', 'r') as f:
        data_top_10 = f.read()
    with open('longest_name.csv', 'r') as f:
        longest_name = f.read()
    with open('airflow_position.csv', 'r') as f:
        airflow_position = f.read()
    date = ds

    print(f'Top 10 domains zones for {date}')
    print(data_top_10)

    print(f'Longest name of domain for {date}')
    print(longest_name)

    print(f'Airflow position for {date} is')
    print(airflow_position)




default_args = {
    'owner': 'a-novikov-21',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2022, 6, 6),
}
schedule_interval = '0 10 * * *'

dag = DAG('a-novikov-21_lesson2', default_args=default_args, schedule_interval=schedule_interval)

t1 = PythonOperator(task_id='get_data',
                    python_callable=get_data,
                    dag=dag)

t2 = PythonOperator(task_id='get_top_10',
                    python_callable=get_top_10,
                    dag=dag)
                       
t2_longest = PythonOperator(task_id='get_longest_name',
                    python_callable=get_longest_name,
                    dag=dag)

t2_airflow = PythonOperator(task_id='get_airflow_position',
                    python_callable=get_airflow_position,
                    dag=dag)


t3 = PythonOperator(task_id='print_data',
                    python_callable=print_data,
                    dag=dag)

t1 >> [t2, t2_longest, t2_airflow] >> t3


<Task(PythonOperator): print_data>